In [12]:
import pandas as pd
import numpy as np
import string
from collections import Counter
from nltk.corpus import stopwords


In [23]:
train_dataset=pd.DataFrame(pd.read_csv('train2.csv'))
dev_dataset=pd.read_csv('dev.csv')
train_dataset.head()
stopWords = stopwords.words("english")


In [24]:
#Clean data, remove punctuation and lower the text
def clean_data(x):
    x=x['Comment'].lower()
    x=x.translate(x.maketrans(string.punctuation,' ' * len(string.punctuation)))
    return x.replace('  ',' ').strip()


def vectorize(x):
    tokens=Counter(x['Comment'].split())
    for item in words:
        x[item]=0
        if tokens[item]>0:
            x[item]=tokens[item]
    return x

In [68]:
#Use Bernoulli

#Clean data
train_dataset['Comment']=train_dataset.apply(clean_data, axis=1)
#train_dataset.head()
#Create a set of all the words(even stop words) occurring in the comments
words=set()

#train_dataset.apply(lambda x: words.update(x['Comment'].split()),axis=1)
#Dropping stop words
train_dataset.apply(lambda x: words.update(word for word in x['Comment'].split() if word not in stopWords),axis=1)


#Removing this word since it's a part of the column
words-={'Insult'}

print('Total distinct words in the comment is',len(words))
#train_dataset
vector=train_dataset.apply(vectorize, axis=1)
feature_matrix=vector.drop(['Comment'],axis=1)#.transpose()

agg_feature_matrix=feature_matrix.groupby(['Insult']).sum().T
agg_feature_matrix.head()

Total distinct words in the comment is 16


Insult,0,1
dad,0,1
shut,0,1
friends,0,1
fuck,0,2
burned,0,1


In [70]:
#Aggregations

total=agg_feature_matrix.sum(axis=0)
tot_0=total[0]
tot_1=total[1]
totalwords=tot_1+tot_0

print('Count of 1\'s=',tot_1,',and Count of 0\'s is ',tot_0,',Total Count is ',totalwords)
prob_0=tot_0/totalwords #Probabilty of 0, ie. not insult
prob_1=tot_1/totalwords #Probabilty of 1, ie. insult
print('Probability of 1\'s=',prob_1,',and Probability of 0\'s is ',prob_0)

agg_feature_matrix['word_prob_gvn1']=agg_feature_matrix.apply(lambda x: x[1]/tot_1,axis=1)
agg_feature_matrix['word_prob_tot']=agg_feature_matrix.apply(lambda x: (x[1]+x[0])/totalwords,axis=1)
#agg_feature_matrix.T
proba_matrix=agg_feature_matrix.T.iloc[[2,3]].reset_index()
proba_matrix.to_csv('prob_out.csv',index=False)

Count of 1's= 10 ,and Count of 0's is  8 ,Total Count is  18
Probability of 1's= 0.5555555555555556 ,and Probability of 0's is  0.4444444444444444


In [ ]:
#P(1|Words)=(P(Words|1)*P(1))/P(Words)
#P(1|Words)=P(word1|1)*P(word2|1)*P(word3|1)...*P(1)/(P(word1)*P(word2)*P(word3)...)

#probability of 1, given words=((probability of words given 1)*probability of words)/(probabilty of words)

In [71]:
prob_out=pd.read_csv('prob_out.csv')
prob_out

,Insult,dad,shut,friends,fuck,burned,understand,oranges,faggot,seems,apples,stake,rest,point,really,xa0,mixing
0,word_prob_gvn1,0.100000,0.100000,0.100000,0.200000,0.100000,0.000000,0.100000,0.100000,0.000000,0.000000,0.100000,0.100000,0.000000,0.000000,0.000000,0.000000
1,word_prob_tot,0.055556,0.055556,0.055556,0.111111,0.055556,0.055556,0.111111,0.055556,0.055556,0.055556,0.055556,0.055556,0.055556,0.055556,0.055556,0.055556


In [185]:
#Pass df of comment
def test_data(x):
    #print(x)
    comment=clean_data(x)
    #print(comment)
    net_prob=float(1.0)
    
    words=[word for word in comment.split() if word not in stopWords]
    for word in words:
        wordprob_gvn1=prob_out[word][[0]]
        prob_word=prob_out[word][[1]]
        #print(wordprob_gvn1,wordprob_gvn1)
        net_prob=(net_prob*wordprob_gvn1)
        print('Multi',net_prob,wordprob_gvn1)
        net_pro=net_prob/prob_word
        print('Divide',net_prob,prob_word)
    return net_prob*prob_1
        
        

In [186]:
#Doubts
#How to make model ot of it?
# Is it just like method call?
#What if input has same word multiple times?
# Remove index while saving csv


#if prob_out[word][[0]]>0:
         #   net_prob=net_prob*prob_out[word][[0]]
        
        #print('Net prob2',net_prob)
        #if prob_out[word][[1]]>0:
         #   net_prob=net_prob*prob_out[word][[1]]
        #print('Net prob3',net_prob)
    #print(net_prob*prob_1)    

In [187]:
test=pd.read_csv('train2.csv')

test.apply(test_data,axis=1)

Multi 0    0.2
Name: fuck, dtype: float64 0    0.2
Name: fuck, dtype: float64
Divide 0    0.2
Name: fuck, dtype: float64 1    0.111111
Name: fuck, dtype: float64
Multi 0    0.02
dtype: float64 0    0.1
Name: dad, dtype: float64
Divide 0    0.02
dtype: float64 1    0.055556
Name: dad, dtype: float64
Multi 0    0.002
dtype: float64 0    0.1
Name: oranges, dtype: float64
Divide 0    0.002
dtype: float64 1    0.111111
Name: oranges, dtype: float64
Multi 0    0.2
Name: fuck, dtype: float64 0    0.2
Name: fuck, dtype: float64
Divide 0    0.2
Name: fuck, dtype: float64 1    0.111111
Name: fuck, dtype: float64
Multi 0    0.02
dtype: float64 0    0.1
Name: dad, dtype: float64
Divide 0    0.02
dtype: float64 1    0.055556
Name: dad, dtype: float64
Multi 0    0.002
dtype: float64 0    0.1
Name: oranges, dtype: float64
Divide 0    0.002
dtype: float64 1    0.111111
Name: oranges, dtype: float64


,0
0,0.001111
